# Deliverable 2

## By:
## Ahmed Mozammil Iqbal (Roll no. 24100085)
## Javeria Siddique (Roll no. 24100246)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
!pip install ujson
import ujson

In [2]:
class TicTacToe:
    def __init__(self):
        self.board = [[" " for _ in range(4)] for _ in range(4)]
        self.current_player = -1


    def check_draw(self):
        for row in self.board:
            if " " in row:
                return False
        return True

    def print_board(self):
        # Prints a GUI-like representation of the board
        print("┌───┬───┬───┬───┐")
        for i, row in enumerate(self.board):
            print("│ " + " │ ".join(row) + " │")
            if i < 3:
                print("├───┼───┼───┼───┤")
        print("└───┴───┴───┴───┘")

    def check_winner(self, player):
        for row in self.board:
            if all([cell == player for cell in row]):
                return True
        for col in range(4):
            if all([self.board[row][col] == player for row in range(4)]):
                return True
        if all([self.board[i][i] == player for i in range(4)]) or all(
            [self.board[i][3 - i] == player for i in range(4)]
        ):
            return True
        return False

    def step(self, state):
        row = int(state / 4)
        col = int(state % 4)

        current_player_symbol = " "
        if self.current_player == -1:
            current_player_symbol = "X"
        else:
            current_player_symbol = "O"

        if self.board[row][col] == " ":
            self.board[row][col] = current_player_symbol

        if self.check_winner("O"):
            return self.board, self.current_player, True, -1
        elif self.check_winner("X"):
            return self.board, self.current_player, True, 1
        elif self.check_draw():
            return self.board, self.current_player, True, 0

        self.current_player *= -1

        return self.board, self.current_player, False, 0

In [3]:
# Initialized as a random policy for player 1

# def policy_player1(board):

#     possible_actions = []

#     for i in range(4):
#         for j in range(4):
#             if board[i][j] == " ":
#                 possible_actions.append(i*4 + j)


#     return random.choice(possible_actions)



# # Initialized as a random policy for player 2
# def policy_player2(board):

#     possible_actions = []

#     for i in range(4):
#         for j in range(4):
#             if board[i][j] == " ":
#                 possible_actions.append(i*4 + j)



#     return random.choice(possible_actions)

# Q LEARNING (training)

In [2]:
#Helper functions

#Generate all 4 by 4 states

# We generate all the states possible for the algorithm. We have used meshgrid as it is more optimized than our previous recursive definition
def generate_all_states():
    values = [-1, 0, 1]
    combinations = np.array(np.meshgrid(*([values] * 16))).T.reshape(-1, 16)
    return combinations.tolist()

# Removes all impossible states
def remove_impossible_states(allStates):
    allStates = np.array(allStates)

    # Count occurrences of -1 and 1 along axis 1
    numX = np.count_nonzero(allStates == -1, axis=1)
    numO = np.count_nonzero(allStates == 1, axis=1)

    # This makes a mask of the states which should be allowed
    mask = np.logical_or(numX == numO, np.logical_or(numO == numX - 1, np.logical_and(numX == 0, numO == 0)))

    # We apply the mask to filter out all illegal states
    reducedStates = allStates[mask]


    # We remove the states where both players are winning
    finalStates = [tuple(state) for state in reducedStates if not (player_wins(state, -1) and player_wins(state, 1))]

    print("all states: ", len(allStates))
    print("final states: ", len(finalStates))

    return finalStates

# player is either -1 (X) or 1 (O)
# we check if X wins or O wins with this function, will be used to get the final derived states
# (We split it in multiple If statement for better readibility)
def player_wins(state, player):
    # Check rows
    if state[0] == player and state[1] == player and state[2] == player and state[3] == player:
        return True
    elif state[4] == player and state[5] == player and state[6] == player and state[7] == player:
        return True
    elif state[8] == player and state[9] == player and state[10] == player and state[11] == player:
        return True
    elif state[12] == player and state[13] == player and state[14] == player and state[15] == player:
        return True

    # Check columns
    elif state[0] == player and state[4] == player and state[8] == player and state[12] == player:
        return True
    elif state[1] == player and state[5] == player and state[9] == player and state[13] == player:
        return True
    elif state[2] == player and state[6] == player and state[10] == player and state[14] == player:
        return True
    elif state[3] == player and state[7] == player and state[11] == player and state[15] == player:
        return True

    # Check diagonals
    elif state[0] == player and state[5] == player and state[10] == player and state[15] == player:
        return True
    elif state[3] == player and state[6] == player and state[9] == player and state[12] == player:
        return True
    else:
        return False

# Given a state this will find the next possible states
def next_possible_states(state, playeraction):
    zero_indexes = np.where(np.array(state) == 0)[0]
    nextstates = [np.copy(state) for _ in zero_indexes]

    for i, pos_nextaction in enumerate(zero_indexes):
        nextstates[i][pos_nextaction] = playeraction

    return nextstates, zero_indexes

In [5]:
# all_states=remove_impossible_states(generate_all_states())
# print("All states (allowed + not allowed) generated")
# all_states = np.array(all_states)
# np.save("all_states.npy", all_states)

In [7]:
# # Initialization
# #Initialising all the tables

# # For computing policy table
# policy_table={}
# for state in all_states:
#     policy_table[state]=None
# print("policy_table initialized")

# with open('policy_table.txt', 'w') as file:
#     file.write(ujson.dumps(policy_table))

# # Reward Table
# reward_table={}
# for state in all_states:
#     if player_wins(state, -1):
#         reward_table[state]=1
#     elif player_wins(state, 1):
#         reward_table[state]=-1
#     else:
#         reward_table[state]=0
# print("reward_table initialized")

# with open('reward_table.txt', 'w') as file:
#     file.write(ujson.dumps(reward_table))

# # Turns Table: says whose turn it is at each instance
# turns_table = {}
# for state in all_states:
#     if np.all(np.array(state) == 0):
#         turns_table[state] = -1
#     elif np.count_nonzero(np.array(state) == -1) > np.count_nonzero(np.array(state) == 1):
#         turns_table[state] = 1
#     else:
#         turns_table[state] = -1
# print("turns_table initialized")

# with open('turns_table.txt', 'w') as file:
#     file.write(ujson.dumps(turns_table))

# # Stores the next state of all the states
# # convert to dictionary of state: {action:nextstate, action2:....}
# next_state_from_action={}
# action_locations={} #stores places of next action corresponding to the next_states in question
# all_next_states={}
# for state in all_states:
#     whose_turn=turns_table[str(state)]
#     all_next_states[state], action_locations[state] =  next_possible_states(state, whose_turn)

#     next_state_from_action[state]={action:state_from_action for action, state_from_action in zip(action_locations[state], all_next_states[state])}
# print("next_states and action_locations initialised")


# str_action_locations = {}
# for key, value in action_locations.items():
#     str_action_locations[str(key)] = str(tuple(value))

# with open('action_locations.txt', 'w') as file:
#     file.write(ujson.dumps(str_action_locations))

# str_next_state_from_action = {}
# for key in next_state_from_action.keys():
#     str_next_state_from_action[str(key)] = {}
#     for internal_key in next_state_from_action[key].keys():
#         str_next_state_from_action[str(key)][internal_key] = str(tuple(next_state_from_action[key][internal_key]))

# with open('next_state_from_action.txt', 'w') as file:
#     file.write(ujson.dumps(str_next_state_from_action))

# #Q table: action is 0, 1, 2, 3,....,16
# Q_table={} #state: {action1: value of this state, action2: value of next state2, action3: value of next state3....} according to next_states table
# for state in all_states:
#     actions = {action: 0 for action in action_locations[state]}
#     Q_table[state]=actions
#     pass
# print("Q table initialised")

# with open('q_table.txt', 'w') as file:
#     file.write(ujson.dumps(Q_table))

# Value_table={}
# for state in all_states:
#     Value_table[state]=0
# print("Value table initialised")

# with open('value_table.txt', 'w') as file:
#     file.write(ujson.dumps(Value_table))


# # Terminal states for the game:

# # All states mapped to true or false if terminal or not
# Terminal_states={}
# for state in all_states:
#     if player_wins(state, -1) or player_wins(state, 1):
#         Terminal_states[state]=True
#         Q_table[state]={}
#         all_next_states[state]=[]
#         action_locations[state]=[]
#         next_state_from_action[state]={}
#     elif Q_table[state]=={}:
#         Terminal_states[state]=True
#     else:
#         Terminal_states[state]=False
# print("Terminal states initialised")

# with open('terminal_states.txt', 'w') as file:
#     file.write(ujson.dumps(Terminal_states))

In [22]:
all_states = np.load("all_states.npy", allow_pickle=True) 
all_states = [tuple(state) for state in all_states]
policy_table = ujson.load(open('policy_table.txt', 'r'))
print(len(policy_table))
reward_table = ujson.load(open('reward_table.txt', 'r'))
print(len(reward_table))
turns_table = ujson.load(open('turns_table.txt', 'r'))
print(len(turns_table))
next_state_from_action = ujson.load(open('next_state_from_action.txt', 'r'))
print(len(next_state_from_action))
Q_table = ujson.load(open('q_table.txt', 'r'))
print(len(Q_table))
Value_table = ujson.load(open('value_table.txt', 'r'))
print(len(Value_table))
Terminal_states = ujson.load(open('terminal_states.txt', 'r'))
print(len(Terminal_states))
action_locations = ujson.load(open('action_locations.txt', 'r'))
print(len(action_locations))

10110713


In [28]:
#The Q learning function (also needs edits)
def Qlearning(gamma, alpha, epsilon, total_actions, reward_table, value_table, Q_table, policy_table, epochs):
  count = 0
  initial_epsilon = epsilon
  min_epsilon = 0.1
  min_alpha = 0.01
  decay_rate = 0.001

  for _ in range(epochs):
    state=(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    if count % 10000 == 0:
      print(count)
    while True:
      if np.random.rand() < epsilon:
        num_actions = action_locations[str(state)].strip('()').split(',')
        num_actions = [item.replace(" ", "") for item in num_actions]
        num_actions = [action for action in num_actions if action != '']
        action = random.choice(num_actions) #at times it may choose a random policy
      else:
        action = max(Q_table[str(state)], key=Q_table[str(state)].get) #but otherwise it chooses from the learned policies (this is argmax with dictionaries)
      next_state=next_state_from_action[str(state)][str(action)] #this is the next state after taking the action
      max_value = 0
      min_value = 0
      if Terminal_states[str(next_state)] == False:
        max_value = max(Q_table[str(next_state)].values())   
        min_value = min(Q_table[str(next_state)].values())      
      reward = reward_table[str(next_state)]

      if turns_table[str(state)]==-1:
        Q_table[str(state)][str(action)] += alpha * (reward + gamma * max_value - Q_table[str(state)][str(action)])
      else:
        Q_table[str(state)][str(action)] += alpha * (reward + gamma * min_value - Q_table[str(state)][str(action)])

      state=next_state

      # we exit if state is one of the terminal states
      if Terminal_states[str(state)]==True:
        break
    count += 1
    epsilon = (initial_epsilon - min_epsilon) * max(0, ((epochs - count) / epochs)) + min_epsilon
    if count % 1000 == 0:
      alpha = max(min_alpha, alpha * (1 - decay_rate))

  return Q_table

In [30]:
Q_table = Qlearning(0.9, 0.9, 0.9, 16, reward_table, Value_table, Q_table, policy_table, 1000000)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000


In [31]:
for state in Q_table.keys():
    if Q_table[state] == {}:
        continue
    if turns_table[str(state)] == -1:
        policy_table[str(state)] = max(Q_table[state], key=Q_table[state].get)
    else:
        policy_table[str(state)] = min(Q_table[state], key=Q_table[state].get)


In [32]:
with open('policy_table.txt', 'w') as file:
    file.write(ujson.dumps(policy_table))

In [36]:
# # Functions to save our policy to a file for easy retrieval and access
# # For evaluation

def load_policies():
    # Load the policy table from the file
    with open('policy_table.txt', 'r') as file:
        policy_table = ujson.load(file)
    return policy_table
    
policy = load_policies()

len(policy)

10110713

In [33]:
# # function to retrieve policy from the file
def get_policy(state):
    state = np.array(state).flatten()
    state = state.tolist()
    for i in range(len(state)):
        if state[i] == " ":
            state[i] = 0
        elif state[i] == "X":
            state[i] = -1
        elif state[i] == "O":
            state[i] = 1
    print("new state: ", state)
    state=str(tuple(state))
    return policy[state]

# Application of code starts here (was part of already given boiler code):

In [34]:
def play_one_game():
    tictactoe = TicTacToe()
    terminated = 0
    board = [[" " for _ in range(4)] for _ in range(4)]
    for i in range(8):
        for turn in [-1, 1]:
            action = 0
            if turn == -1:
                action = get_policy(board)
            else:
                action = get_policy(board)
            board, player, terminated, reward = tictactoe.step(int(action))
            # Uncomment this if you want to see the board
            tictactoe.print_board()
            if terminated:
                break
    return -1*reward # This is the player who won

In [ ]:
play_one_game()

In [ ]:
# def run_alternating_games(games=10):
#     results = []
#     for i in range(games):
#         for j in range(2):
#             if j==0:
#                 winner = play_one_game(policy_player1, policy_player2)

#                 match winner:
#                     case -1:
#                         results.append(1)
#                     case 1:
#                         results.append(2)
#                     case 0:
#                         results.append(0)

#             if j==1:
#                 winner = play_one_game(policy_player2, policy_player1)

#                 match winner:
#                     case -1:
#                         results.append(2)
#                     case 1:
#                         results.append(1)
#                     case 0:
#                         results.append(0)


#     return results

In [ ]:
# results = run_alternating_games(1000)
# print("Draws: ", results.count(0))
# print("Player 1 Wins:", results.count(1))
# print("Player 2 Wins:", results.count(2))

I have created two functions that randomly select any action from the available actions from the board. Your team will have to create such a function that outputs the optimal action given a particular board state. This a similar kind of code I will be using on competition day when your function will play against an opponent's functions for perhaps a 1000 games.

I will pass your and your opponent's function into the run alternating games function for maybe 1000 games to see who won more games. That person will be the winner of the match. I think it's a reliable method to compare policies. Run them by each for 1000s of games and see what policy wins the most games.

You have to solve this part using **Q Learning**